In [1]:
# Imports
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from flourish_child.models import Appointment

In [7]:
def switch_child_to_cohortb_quart(cv):
    #Maternal Visit
#     mv.save()

    cold_q_apps = None
    cq_ons = None

    try:
        cq_ons = OnScheduleChildCohortBQuarterly.objects.get(
            subject_identifier=cv.subject_identifier)
    except OnScheduleChildCohortBQuarterly.DoesNotExist:
        print('whoa!', cv.subject_identifier)
    else:
        cold_q_apps = Appointment.objects.filter(subject_identifier=cq_ons.subject_identifier,
                                             schedule_name=cq_ons.schedule_name,
                                             visit_code_sequence=0)
    
    cvs = cv.schedule_name.split('_')
    cfschedule_name = '_'.join([cvs[0], cvs[1], cvs[2].replace('enrol', 'quart'), cvs[3]])
        
    #check new enrollment onschedule model
    print(">>>>>>>>>>>>>", cfschedule_name)
    try:
        cons = OnScheduleChildCohortCQuarterly.objects.get(
            subject_identifier=cv.subject_identifier,
            schedule_name=cfschedule_name)
    except:
        print('hmmmmm!', cv.subject_identifier)
    else:
        print('Modify newly generated child quart appointments')
        
        if cold_q_apps:
            for appt in Appointment.objects.filter(schedule_name=cons.schedule_name,
                                               subject_identifier=cons.subject_identifier):
                cap = cold_q_apps.get(visit_code=appt.visit_code)
                
                try:
                    v1 = ChildVisit.objects.get(appointment=cap)
                    v1.appointment = appt
                    v1.save()
                except:
                    pass
                
                appt.user_modified = cap.user_modified
                appt.appt_status = cap.appt_status
                appt.appt_reason = cap.appt_reason
                appt.appt_datetime = cap.appt_datetime

                appt.save()

            ssh = SubjectScheduleHistory.objects.get(subject_identifier=cold_q_apps[0].subject_identifier,
                                                           schedule_name=cold_q_apps[0].schedule_name)
            ssh.delete()

            cold_q_apps.delete()
            cq_ons.delete()


In [3]:
def switch_child_to_cohortb_enroll(subject_identifier, new_cohort):
    #Change cohort in consent
    child_dummy_consent = ChildDummySubjectConsent.objects.filter(
        subject_identifier=subject_identifier).latest('consent_datetime')
    child_dummy_consent.cohort = new_cohort
    child_dummy_consent.save()

    cold_apps = None
    ca_ons = None

    try:
        ca_ons = OnScheduleChildCohortBEnrollment.objects.get(
            subject_identifier=child_dummy_consent.subject_identifier)
    except OnScheduleChildCohortBEnrollment.DoesNotExist:
        print('whoa!', child_dummy_consent.subject_identifier)
    else:
        cold_apps = Appointment.objects.filter(subject_identifier=ca_ons.subject_identifier,
                                              schedule_name=ca_ons.schedule_name,
                                              visit_code_sequence=0)

    #check new enrollment onschedule model
    try:
        cons = OnScheduleChildCohortCEnrollment.objects.get(
            subject_identifier=child_dummy_consent.subject_identifier)
    except:
        print('waittttt!', child_dummy_consent.subject_identifier)
    else:
        print('Modify newly generated enrol appointments')
        
        print(cold_apps)
        
        if cold_apps:
            new_apps = Appointment.objects.filter(schedule_name=cons.schedule_name,
                                                  subject_identifier=cons.subject_identifier)
            for a in new_apps:

                ap = cold_apps.get(visit_code=a.visit_code)
                
                print("?????????????", ap)

                try:
                    v1 = ChildVisit.objects.get(appointment=ap)
                    v1.appointment = a
                    v1.save()
                except:
                    pass
    
                a.user_modified = ap.user_modified
                a.appt_status = ap.appt_status
                a.appt_reason = ap.appt_reason
                a.appt_datetime = ap.appt_datetime

                a.save()

            try:
                ssh = SubjectScheduleHistory.objects.get(subject_identifier=cold_apps[0].subject_identifier,
                                                               schedule_name=cold_apps[0].schedule_name)
            except:
                pass
            else:
                ssh.delete()
            
            cold_apps.delete()
            ca_ons.delete()
            
            try:
                initial_cv = ChildVisit.objects.filter(subject_identifier=new_apps[0].subject_identifier,
                                                         schedule_name=new_apps[0].schedule_name).order_by(
                    'report_datetime')[0]
            except ChildVisit.DoesNotExist:
                pass
            else:
                switch_child_to_cohortb_quart(initial_cv)



In [11]:
# pids_2_move = ['B142-040990444-7-10']
#     'B142-040990450-4-10',]
# pids_2_move = ['B142-040990343-1-10', 'B142-040990272-2-10', 'B142-040990161-7-10', 'B142-040990957-8-10']
pids_2_move = ['B142-040990976-8-10', 'B142-040990639-2-10', 'B142-040990771-3-10', 'B142-040990790-3-10',
               'B142-040991005-5-10', 'B142-040990950-3-10', 'B142-040990766-3-10', 'B142-040990849-7-10']

In [12]:
pids_2_move

['B142-040990735-8-10',
 'B142-040990657-4-10',
 'B142-040990584-0-10',
 'B142-040990444-7-10',
 'B142-040990467-8-10']

In [13]:
for pid in pids_2_move:
    switch_child_to_cohortb_enroll(pid, 'cohort_c')

Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000.0>]>
????????????? 2000.0
>>>>>>>>>>>>> child_b_quart_schedule1
Modify newly generated child quart appointments
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000.0>]>
????????????? 2000.0
>>>>>>>>>>>>> child_b_quart_schedule1
Modify newly generated child quart appointments
Modify newly generated enrol appointments
<QuerySet [<Appointment: 2000.0>]>
????????????? 2000.0
>>>>>>>>>>>>> child_b_quart_schedule1
Modify newly generated child quart appointments
whoa! B142-040990444-7-10
Modify newly generated enrol appointments
None
whoa! B142-040990467-8-10
Modify newly generated enrol appointments
None
